In [1]:
import geopandas as gpd
import pandas as pd

import mapillary as mly
#import json

import glob


import gzip

from datetime import datetime



In [2]:


## TRAFFFIC SIGNS
# https://www.mapillary.com/developer/api-documentation/traffic-signs?locale=

# Unterschiedliche Verkehrszeichen in Deutschland für Radwege

# https://trafficsigns.osm-verkehrswende.org/DE?signs=DE:237
# Radweg
de_237="regulatory--bicycles-only--g1"

# https://trafficsigns.osm-verkehrswende.org/DE?signs=DE:240
# Gemeinsamer Geh- und Radweg
de_240="regulatory--shared-path-pedestrians-and-bicycles--g1"

# https://trafficsigns.osm-verkehrswende.org/DE?signs=DE:241-30
# Getrennter Rad- und Gehweg
de_241_30="regulatory--dual-path-bicycles-and-pedestrians--g1"

# https://trafficsigns.osm-verkehrswende.org/DE?signs=DE:241-31
# Getrennter Rad- und Gehweg
de_241_31="regulatory--dual-path-pedestrians-and-bicycles--g1"




#_________________

# # https://trafficsigns.osm-verkehrswende.org/DE?signs=DE:1022-10
# # Radfahrer frei
de_1022_10=  "complementary--except-bicycles--g1" 


# # https://trafficsigns.osm-verkehrswende.org/DE?signs=DE:1000-33
# # Radverkehr im Gegenverkehr
de_1000_33=  "complementary--bike-route--g1" 



# Find all parquet files in the tile_cache folder (per state)
parquet_files = glob.glob("../../output/*.parquet")
# Extract and print the dates from the filenames
dates = [f.split("_")[-1].replace(".parquet", "") for f in parquet_files]
# Drop all duplicates in the list
dates = list(set(dates))
print("Dates found in filenames:", dates)
# Read all files and concatenate into one GeoDataFrame
mapillary_trafficsigns = gpd.GeoDataFrame(
    pd.concat([gpd.read_parquet(f) for f in parquet_files], ignore_index=True),
    crs=gpd.read_parquet(parquet_files[0]).crs
)






# Schilder in Germany
mapillary_trafficsigns_cw=mapillary_trafficsigns[mapillary_trafficsigns.value.isin([de_237, de_240, de_241_30, de_241_31, de_1022_10, de_1000_33])]

mapillary_trafficsigns_cw=mapillary_trafficsigns_cw.drop_duplicates(subset=['id'])

mapillary_trafficsigns_cw = mapillary_trafficsigns_cw.drop(columns=["tile_x", "tile_y"])


mapillary_trafficsigns_cw


# value_VZ DE:237
# 

Dates found in filenames: ['2025-10-11']


,geometry,first_seen_at,id,last_seen_at,value
2,POINT (8.29728 54.87519),2016-09-06,1820694428091863,2016-09-06,regulatory--shared-path-pedestrians-and-bicycl...
30,POINT (8.30306 54.90148),2022-08-31,812432256424826,2022-08-31,complementary--except-bicycles--g1
34,POINT (8.29949 54.89374),2016-09-06,533889454272406,2016-09-06,regulatory--bicycles-only--g1
40,POINT (8.29982 54.88667),2025-06-27,1098704175663599,2025-06-27,regulatory--shared-path-pedestrians-and-bicycl...
44,POINT (8.29988 54.88679),2025-06-27,2217948735386355,2025-06-27,regulatory--shared-path-pedestrians-and-bicycl...
...,...,...,...,...,...
6297807,POINT (8.96308 53.05755),2018-03-18,251362246774370,2018-03-18,regulatory--dual-path-bicycles-and-pedestrians...
6297829,POINT (8.964 53.05729),2025-08-16,1462451174804015,2025-08-16,regulatory--dual-path-pedestrians-and-bicycles...
6297834,POINT (8.96359 53.05766),2025-08-16,3058286184362909,2025-08-16,regulatory--dual-path-bicycles-and-pedestrians...
6297838,POINT (8.96407 53.0572),2017-04-17,129610735792136,2022-04-11,regulatory--dual-path-bicycles-and-pedestrians...


In [3]:
#### Verearbeitung fuer export fuer radinfra.de

In [4]:
mapillary_trafficsigns_cw=mapillary_trafficsigns_cw[mapillary_trafficsigns_cw["last_seen_at"] > "2023-01-01"].copy()
len(mapillary_trafficsigns_cw)

116462

In [5]:

def calc_days_between(row):
    try:
        d1 = datetime.strptime(row['first_seen_at'], "%Y-%m-%d")
        d2 = datetime.strptime(row['last_seen_at'], "%Y-%m-%d")
        return (d2 - d1).days
    except Exception:
        return None

mapillary_trafficsigns_cw["delta_days_seen"] = mapillary_trafficsigns_cw.apply(calc_days_between, axis=1)

In [9]:
mapillary_trafficsigns_cw["Hinweis"] = mapillary_trafficsigns_cw["delta_days_seen"].apply(
    lambda x: "Nur einmalig detektiert, ggf. temporär wie z.B. Baustelle." if x == 0 else ""
)

In [11]:
mapillary_trafficsigns_cw = mapillary_trafficsigns_cw.drop(columns=["delta_days_seen"])


In [12]:
# Mapping dictionary
value_to_ts = {
    'regulatory--dual-path-pedestrians-and-bicycles--g1': "DE:241",
    'regulatory--dual-path-bicycles-and-pedestrians--g1': "DE:241",
    'regulatory--bicycles-only--g1': "DE:237",
    'regulatory--shared-path-pedestrians-and-bicycles--g1': "DE:240",
    
    'complementary--except-bicycles--g1': "DE:1022-10",
    'complementary--bike-route--g1': "DE:1000-33"
}

value_to_tsd = {
    'regulatory--dual-path-pedestrians-and-bicycles--g1': 'Getrennter Geh- und Radweg',
    'regulatory--dual-path-bicycles-and-pedestrians--g1': 'Getrennter Geh- und Radweg',
    'regulatory--bicycles-only--g1': 'Radweg',
    'regulatory--shared-path-pedestrians-and-bicycles--g1': 'Gemeinsamer Geh- und Radweg',
    'complementary--except-bicycles--g1': 'Radfahrer frei',
    'complementary--bike-route--g1': 'Radverkehr im Gegenverkehr'
}

# Add new column based on mapping
mapillary_trafficsigns_cw["traffic_sign"] = mapillary_trafficsigns_cw["value"].map(value_to_ts)
mapillary_trafficsigns_cw["traffic_sign_description"] = mapillary_trafficsigns_cw["value"].map(value_to_tsd)

In [13]:
mapillary_trafficsigns_cw

,geometry,first_seen_at,id,last_seen_at,value,traffic_sign,traffic_sign_description,Hinweis
40,POINT (8.29982 54.88667),2025-06-27,1098704175663599,2025-06-27,regulatory--shared-path-pedestrians-and-bicycl...,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
44,POINT (8.29988 54.88679),2025-06-27,2217948735386355,2025-06-27,regulatory--shared-path-pedestrians-and-bicycl...,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
48,POINT (8.29996 54.88709),2025-06-27,1270110387694692,2025-06-27,regulatory--shared-path-pedestrians-and-bicycl...,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
61,POINT (8.30096 54.88757),2025-06-27,3280755865457874,2025-06-27,regulatory--shared-path-pedestrians-and-bicycl...,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
63,POINT (8.30098 54.88761),2025-06-27,1274089467293020,2025-06-27,complementary--bike-route--g1,DE:1000-33,Radverkehr im Gegenverkehr,"Nur einmalig detektiert, ggf. temporär wie z.B..."
...,...,...,...,...,...,...,...,...
6297431,POINT (8.90864 53.05984),2025-05-16,1025090599601009,2025-05-16,regulatory--bicycles-only--g1,DE:237,Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
6297782,POINT (8.96325 53.05715),2025-08-16,3058286197696241,2025-08-16,regulatory--dual-path-bicycles-and-pedestrians...,DE:241,Getrennter Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
6297795,POINT (8.9643 53.05719),2025-08-16,1462451184804014,2025-08-16,regulatory--dual-path-bicycles-and-pedestrians...,DE:241,Getrennter Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."
6297829,POINT (8.964 53.05729),2025-08-16,1462451174804015,2025-08-16,regulatory--dual-path-pedestrians-and-bicycles...,DE:241,Getrennter Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B..."


In [15]:
mapillary_trafficsigns_cw = mapillary_trafficsigns_cw[['traffic_sign', 'traffic_sign_description', 'Hinweis',
    'first_seen_at', 'last_seen_at', 'id','value','geometry']]

In [16]:
mapillary_trafficsigns_cw

,traffic_sign,traffic_sign_description,Hinweis,first_seen_at,last_seen_at,id,value,geometry
40,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-06-27,2025-06-27,1098704175663599,regulatory--shared-path-pedestrians-and-bicycl...,POINT (8.29982 54.88667)
44,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-06-27,2025-06-27,2217948735386355,regulatory--shared-path-pedestrians-and-bicycl...,POINT (8.29988 54.88679)
48,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-06-27,2025-06-27,1270110387694692,regulatory--shared-path-pedestrians-and-bicycl...,POINT (8.29996 54.88709)
61,DE:240,Gemeinsamer Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-06-27,2025-06-27,3280755865457874,regulatory--shared-path-pedestrians-and-bicycl...,POINT (8.30096 54.88757)
63,DE:1000-33,Radverkehr im Gegenverkehr,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-06-27,2025-06-27,1274089467293020,complementary--bike-route--g1,POINT (8.30098 54.88761)
...,...,...,...,...,...,...,...,...
6297431,DE:237,Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-05-16,2025-05-16,1025090599601009,regulatory--bicycles-only--g1,POINT (8.90864 53.05984)
6297782,DE:241,Getrennter Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-08-16,2025-08-16,3058286197696241,regulatory--dual-path-bicycles-and-pedestrians...,POINT (8.96325 53.05715)
6297795,DE:241,Getrennter Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-08-16,2025-08-16,1462451184804014,regulatory--dual-path-bicycles-and-pedestrians...,POINT (8.9643 53.05719)
6297829,DE:241,Getrennter Geh- und Radweg,"Nur einmalig detektiert, ggf. temporär wie z.B...",2025-08-16,2025-08-16,1462451174804015,regulatory--dual-path-pedestrians-and-bicycles...,POINT (8.964 53.05729)


In [17]:
# Save cw traffic sings to compressed GeoJSON and update readme

with gzip.open("ts_output/mapillary_trafficsigns_bicycle_latest.geojson.gz", "wt", encoding="utf-8") as f:
    f.write(mapillary_trafficsigns_cw.to_json())

    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Create the content for the README file
    readme_content = f"""
# Traffic Signs Output
This folder contains the output files for mapillary detected traffic signs.  
The output has been created on {current_date}.

Filter:
- newer than 2023-01-01
- signs for cycleways (DE:237, DE:240, DE:241)
- signs for bicycle (DE:1022-10, DE:1000-33)
    """

    # Write the README file
    with open("ts_output/README.md", "w", encoding="utf-8") as readme_file:
        readme_file.write(readme_content)

# Save cw traffic sings to geoparquet
#mapillary_trafficsigns_cw.to_parquet("mapillary_trafficsigns_cw-237-240-241.parquet")